In [ ]:
! pip3 install --upgrade pip
! pip3 install langchain openai psycopg2-binary langchain_experimental langchain-openai

In [ ]:
from langchain import SQLDatabase
from langchain_openai import ChatOpenAI
from langchain_experimental.sql import SQLDatabaseChain
import os
import csv
import psycopg2
from huggingface_hub import HfApi
import sqlite3
import gradio as gr

In [ ]:
DB_USERNAME = os.environ.get("DB_USERNAME")
DB_PASSWORD = os.environ.get("DB_PASSWORD")
DB_HOST = os.environ.get("DB_HOST", "localhost")
DB_PORT = os.environ.get("DB_PORT", "5432")
DB_NAME = os.environ.get("DB_NAME")
OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY")
GPT_MODEL = "gpt-3.5-turbo-0125"
db_type = "none"

## Setup Database

[Allowing remote DB connections](https://blog.devart.com/configure-postgresql-to-allow-remote-connection.html)


```bash
sudo apt install postgresql
sudo -i -u postgres
createdb <db name>
createuser --interactive --pwprompt <username>
psql
GRANT ALL PRIVILEGES ON DATABASE <db name> TO <username>;
\q
exit
sudo nano /etc/postgresql/12/main/pg_hba.conf
sudo systemctl restart postgresql
psql -U <username> -d <db name>
```

## Create tables
```sql
CREATE TABLE travel_category (
    category_id INTEGER PRIMARY KEY,
    main_category VARCHAR(100) NOT NULL,
    category VARCHAR(100) NOT NULL,
    sub_category VARCHAR(100) NOT NULL,
    emission_factor DOUBLE PRECISION NOT NULL,
    transaction_unit VARCHAR(100) NOT NULL
);
```
```sql
CREATE TABLE employee (
    employee_id INTEGER PRIMARY KEY,
    title VARCHAR(100) NOT NULL,
    department VARCHAR(100) NOT NULL
);
```
```sql
CREATE TABLE travel (
    travel_id SERIAL PRIMARY KEY,
    source VARCHAR(100) NOT NULL,
    destination VARCHAR(100) NOT NULL,
    transaction_value DOUBLE PRECISION NOT NULL,
    cost DOUBLE PRECISION NOT NULL,
    date DATE NOT NULL,
    category_id INTEGER REFERENCES travel_category(category_id) ON DELETE CASCADE,
    employee_id INTEGER REFERENCES employee(employee_id) ON DELETE CASCADE,
    transaction_unit VARCHAR(100) NOT NULL
);
```
```sql
CREATE TABLE date (
    date_id VARCHAR(100) PRIMARY KEY,
    quarter INTEGER NOT NULL,
    month INTEGER NOT NULL,
    year INTEGER NOT NULL
);
```

## Load the data

In [ ]:
if db_type == "postgresql":
    conn = psycopg2.connect(
        dbname=DB_NAME, user=DB_USERNAME, password=DB_PASSWORD, host=DB_HOST
    )
    cursor = conn.cursor()
else:
    conn = sqlite3.connect("carbon-emissions.db")
    cursor = conn.cursor()

In [ ]:
def create_table(table_query):
    try:
        cursor.execute(table_query)
        conn.commit()
        print(f"Table creation successfully.")
    except Exception as e:
        conn.rollback()


def load_data_into_postgresql(csv_file, table_name):
    with open(csv_file, "r", newline="") as file:
        reader = csv.reader(file)
        next(reader)
        for row in reader:
            try:
                placeholders = ",".join(["%s"] * len(row))
                sql = f"INSERT INTO {table_name} VALUES ({placeholders})"
                cursor.execute(sql, row)
            except psycopg2.Error as e:
                conn.rollback()
                print(f"Error inserting row: {e}")
                continue
    conn.commit()
    print(f"Data loaded into {table_name} table successfully.")


def load_data_into_sqlite(csv_file, table_name):
    with open(csv_file, "r", newline="") as file:
        reader = csv.reader(file)
        next(reader)
        for row in reader:
            try:
                placeholders = ",".join(["?"] * len(row))
                sql = f"INSERT INTO {table_name} VALUES ({placeholders})"
                cursor.execute(sql, row)
            except Exception as e:
                conn.rollback()
                print(f"Error inserting row: {e}")
                continue
    conn.commit()
    print(f"Data loaded into {table_name} table successfully.")

In [ ]:
create_travel_category_table_query = """
    CREATE TABLE travel_category (
        category_id INTEGER PRIMARY KEY,
        main_category VARCHAR(100) NOT NULL,
        category VARCHAR(100) NOT NULL,
        sub_category VARCHAR(100) NOT NULL,
        emission_factor DOUBLE PRECISION NOT NULL,
        transaction_unit VARCHAR(100) NOT NULL
    );
"""

create_employee_table_query = """
    CREATE TABLE employee (
        employee_id INTEGER PRIMARY KEY,
        title VARCHAR(100) NOT NULL,
        department VARCHAR(100) NOT NULL
    );
"""

create_travel_table_query = """
    CREATE TABLE travel (
        travel_id SERIAL PRIMARY KEY,
        source VARCHAR(100) NOT NULL,
        destination VARCHAR(100) NOT NULL,
        transaction_value DOUBLE PRECISION NOT NULL,
        cost DOUBLE PRECISION NOT NULL,
        date DATE NOT NULL,
        category_id INTEGER REFERENCES travel_category(category_id) ON DELETE CASCADE,
        employee_id INTEGER REFERENCES employee(employee_id) ON DELETE CASCADE,
        transaction_unit VARCHAR(100) NOT NULL
    );
"""

In [ ]:
create_table(create_travel_category_table_query)
create_table(create_employee_table_query)
create_table(create_travel_table_query)

In [ ]:
if db_type == "postgresql":
    load_data_into_postgresql("employee.csv", "employee")
    load_data_into_postgresql("travel_category.csv", "travel_category")
    load_data_into_postgresql("travel.csv", "travel")
else:
    load_data_into_sqlite("employee.csv", "employee")
    load_data_into_sqlite("travel_category.csv", "travel_category")
    load_data_into_sqlite("travel.csv", "travel")

In [ ]:
cursor.close()
conn.close()

## Setup LLM

In [ ]:
if db_type == "postgresql":
    db_uri = f"postgresql+psycopg2://{DB_USERNAME}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}"
else:
    db_uri = "sqlite:///carbon-emissions.db"

db = SQLDatabase.from_uri(db_uri)
llm = ChatOpenAI(api_key=OPENAI_API_KEY, model=GPT_MODEL)
db_chain = SQLDatabaseChain(llm=llm, database=db, verbose=True, top_k=3)

In [ ]:
PROMPT = """ 
Given an input question, first create a syntactically correct Postgresql query without ```sql formatting in the query. 
Run the query then look at the results of the query.
Interpret th results and return an answer.  
The question: {question}
"""

## Run a query

In [ ]:
question = "what is the average carbo emission for 2023?"
response = db_chain.run(PROMPT.format(question=question))

In [ ]:
response

## Deploy to Huggingface

In [ ]:
token = os.getenv("HUGGINFACE_TAKEN")
repo_id = "noahnsimbe/carbon-emissions-chat"
api = HfApi(token=token)
api.upload_file(
    path_or_fileobj="carbon-emissions.db",
    path_in_repo="carbon-emissions.db",
    repo_id=repo_id,
    repo_type="space",
)
api.upload_file(
    path_or_fileobj="requirements.txt",
    path_in_repo="requirements.txt",
    repo_id=repo_id,
    repo_type="space",
)
api.upload_file(
    path_or_fileobj="app.py",
    path_in_repo="app.py",
    repo_id=repo_id,
    repo_type="space",
)
api.upload_file(
    path_or_fileobj="ChatBot-README.md",
    path_in_repo="README.md",
    repo_id=repo_id,
    repo_type="space",
)